In [1]:
import pandas as pd
import numpy as np

In [2]:
#how to determine which encoding to use. Went on python documentation to look at different encodings. Know consequences
df = pd.read_csv('WELLCOME_APCspend2013_forThinkful.csv', encoding='latin1')
clean = df.copy(deep=True)
print(clean.keys())
cost = 'COST (£) charged to Wellcome (inc VAT when charged)'

Index(['PMID/PMCID', 'Publisher', 'Journal title', 'Article title',
       'COST (£) charged to Wellcome (inc VAT when charged)'],
      dtype='object')


In [3]:
#Removing any whitespace and getting rid of duplicates by fixing capitalizations
clean['PMID/PMCID'] = clean['PMID/PMCID'].str.strip()
clean['Journal title'] = clean['Journal title'].str.strip().str.lower().str.capitalize()
clean.Publisher = clean.Publisher.str.strip().str.lower().str.capitalize()

In [4]:
journals = {
    'PLOS One': ['Plos one', 'Plosone'],
    'Acta Crystallographica Section D: Structural Biology': ['Acta crystallographica section d,  biological crystallography','Acta crystallographica section d: biological crystallography', 'Acta crystallographica, section d',
                                            'Acta crystallography d', 'Acta d'],
    'Acta Crystallographica Section F: Structural Biology Communications' : ['Acta crystallographica section f: structural biology and crystallization communications', 'Acta f'],
    'Acta Neuropathologica': ['Acta neuropathologica', 'Acta neuropathol'],
    'Age and Ageing': ['Age and ageing', 'Age'],
    'AIDS': ['Aids', 'Aids journal'],
    'American Journal of Tropical Medicine and Hygiene': ['Am j trop med hyg'],
    'American Journal of Bioethics': ['Am j bioeth']
}

for k, v in journals.items():
    clean.loc[clean['Journal title'].isin(v), 'Journal title'] = k

#clean.loc[clean['Journal title'].str.startswith('A', na=False), 'Journal title'].value_counts()
clean['Journal title'].value_counts().sort_values(ascending=False)

PLOS One                                                  199
Journal of biological chemistry                            53
Neuroimage                                                 29
Nucleic acids research                                     26
Plos pathogens                                             24
Plos genetics                                              24
Proceedings of the national academy of sciences            22
Plos neglected tropical diseases                           20
Nature communications                                      19
Human molecular genetics                                   19
Movement disorders                                         15
Brain                                                      14
Bmc public health                                          14
Journal of neuroscience                                    13
Developmental cell                                         12
Biochemical journal                                        12
Current 

In [5]:
#hash table for the correct publisher values
publish = {
    'ACS': ['Acs', 'Acs (amercian chemical society) publications', 'Acs publications', 'American chemical society', 
            'American chemical society publications'],
    'American Society for Biochemistry and Molecular Biology': ['AMBSB', 'ASBMB Cadmus', 'ASBMB/Cadmus', 'ASBMB/Cenveo Publisher Services', 'ASBMC /CENVEO', 'Ambsb', 'American soc for biochemistry and molecular biology',
             'American society for biochemistry and molecular biolgy', 'American society for biochemistry and molecular biology', 'Asbmb', 'Asbmb cadmus', 'Asbmb/cadmus', 'Asbmb/cenveo publisher services', 'Asbmc /cenveo',
                                                               'The american society for biochemistry and molecular biology', 'The american society for biochemistry and molecular biology, inc'],
    'American Psychological Association': ['American psychiatric association', 'American psychiatric publishing', 
                                           'American psychological association'],
    'American Physiological Society': ['The american physiological society'],
    'American Society of Hematology': ['American society of hematology', 'American society of haematology', 'American society of hamatology'],
    'American Society for Microbiology': ['American society for microbiology', 'American society of microbiology', 'Asm', 'Asm (american society for microbiology)'],
    'Bentham Science Publishers': ['Bentham science publishers', 'Benthan science publishers'],
    'BioMed Central': ['Biomed central', 'Bmc', 'Biomed central limited', 'Biomed central ltd'],
    'The BMJ': ['Bmj', 'Bmj group', 'Bmj publishing group', 'Bmj publishing group ltd', 'British medical journal', 'Bmj journals', 'Bmj publishing group ltd & british thoracic society'],
    'Biophysical Society' : ['Biophysical society', 'Byophysical society'],
    'Cambridge University Press': ['Cambridge university press', 'Cup', 'Cambridge univ press', 'Cambridge uni press', 'Cambridge journals'],
    'Cadmus Journal': ['Cadmus journal services', 'Cadmus', 'Cadmus journal service', 'Camdus journal services', ],
    'Cold Spring Harbor Laboratory Press': ['Cold spring harbor laboratory press', 'Cshlp', 'Cold spring harbor', 'Cold spring habour press', 'Cold spring harbor publications', 'Cold spring harbor press'],
    'Cenovo Publisher Services': ['Cenveo publisher services', 'Cenveo publisher services/asm jv1'],
    'Company of Biologists': ['Company of biologist', 'Company of biologists ltd', 'The company of biolgists', 'The company of biologists',
                             'The company of biologists ltd'],
    'Dartmouth Journal Services': ['Dartmouth journal services', 'Dartmouth journals', 'Darmouth journal services'],
    'Elsevier': ['Elseveier', 'Elseveier science', 'Elsevier (cell press)', 'Elsevier b.v.', 'Elsevier / cell science', 'Elsevier ltd', 'Elsevier/cell press'],
    'Frontiers': ['Frontiers media sa', 'Frontiers media', 'Frontiers research foundation'],
    'The FASEB Journal': ['Faseb', 'Federation of american societies for experimental biology', 'Federation of american societies for experimental biology (faseb)','Federation of the american society of experimental biology'],
    'Future Medicine': ['Future medicine', 'Future medicine ltd'],
    'Hindawi': ['Hindawi publishing corporation', 'Hindawi'],
    'International Union of Crystallography': ['International union of crystallography', 'International union of crystallography (iucr)'],
    'Informa Healthcare Communications': ['Informa healthcare communications', 'Informa healthcare'],
    'Impact Journals': ['Impact journals', 'Impact journals llc'],
    'Institute of Physics': ['Institute of physics', 'Iop publishing'],
    'John Wiley & Sons': ['John wiley', 'John wiley & sons', 'John wiley & sons ltd', 'John wiley & sons inc', 'John wiley and sons ltd', 'John wiley and sons', 'John wiley & sons, inc.'],
    'Landes Bioscience': ['Landes bioscience', 'Landes biosciences'],
    'Mary Ann Liebert': ['Mary ann liebert', 'Mary ann liebert, inc. publishers', 'Mary ann liebert inc'],
    'MYJoVE corp': ['Myjove corporation', 'My jove corporation', 'My jove corp'],
    'MIT Press Open Access': ['Mit press', 'Mit press open access'],
    'Nature': ['Nature publishing group', 'Nature', 'Nature publishing', 'Npg', 'Nature publishing group ltd', 'Nature pg'],
    'National Academy of Sciences': ['ational academy of sciences', 'National academy of sciences, usa', 'National academy of sciences of the united states of america', 'National academy of sciences usa'],
    'Oxford University Press': ['Oxford university press', 'Oup', 'Oxford journals', 'Oxford univ press', 'Oxford university press (oup)', 'Oxford journals (oup)', 'Oxford univesity press'],
    'PLoS': ['Public library of science', 'Plos', 'Plos (public library of science)', 'Plos public library of science'],
    'Portland Press': ['Portland press ltd', 'Portland press'],
    'PNAS': ['Pnas', 'Pnas author publication', 'Proceedings of the national academy of sciences (pnas)'],
    'PubMed': ['Pubmed', 'Pubmed central'],
    'The Royal Society': ['Royal society', 'The royal society'],
    'Royal Society of Chemistry': ['Rsc', 'Royal society of chemistry', 'Royal society for chemistry', 'Rsc publishing'],
    'Springer Publishing': ['Springer', 'Springer science + business media'],
    'Springer-Verlag': ['Springer-verlag gmbh', 'Springer - verlag gmbh', 'Springer-verlag gmbh & ci', 'Springer-verlag gmbh, heidelberger platz 3, d-14197 berlin', 'Springer verlag', 'Springer-veriag gmbh'],
    'Sage Publications': ['Sage', 'Sage publications', 'Sage publications ltd', 'Sage publishers', 'Sage publishing', 'Sage publications inc'],
    'Society for Neuroscience': ['Society for neuroscience' ,'Society for neurosciences', 'Society of neuro sciences', 'Society of neuroscience'],
    'Society for General Microbiology': ['Society for general microbiology', 'Society for genermal microbiology', 'Society of general microbiology'],
    'Society for Leukocyte Biology': ['Society for leukocyte biology', 'Society of leukocyte biology'],
    'Taylor & Francis': ['T&f', 'Taylor & francis', 'Taylor & francis journals', 'Taylor and francis'],
    'Endocrine Society': ['The endocrine socety', 'The endocrine society', 'The endrocrine society'],
    'Sheridan Press': ['The sheridan press', 'Sheridan press'],
    'Royal College of Psychiatrists': ['The royal college of psychiatrists', 'Royal college of psychiatrists'],
    'Wiley': ['Wiley & son', 'Wiley blackwell', 'Wiley online library', 'Wiley subscription services', 'Wiley subscription services  inc.', 'Wiley subscription services inc', 'Wiley subscription serviices inc',
             'Wiley vch', 'Wiley-blackwell', 'Wiley-blackwell, john wiley & sons', 'Wiley-vch', 'Wiley/blackwell', 'Wliey-blackwell'],
    'Wolters Kluwer': ['Wolters kluwer', 'Wolters kluwer health', 'Wolters kluwer n.v./lippinott', 'Wolters kluwers']
          }

for k, v in publish.items():
    clean.loc[clean.Publisher.isin(v), 'Publisher'] = k
#Use this to go through all the publishers alphabetically in order to combine duplicates
clean.loc[clean['Publisher'].str.startswith('W', na=False), 'Publisher'].value_counts().sort_index()

Wiley             244
Wolters Kluwer     12
Name: Publisher, dtype: int64

In [6]:
#key error and type error. possibly due to mixed type inside 
#start off by splitting series into a new dataframe with 2 new series. Since the $ is at the end, everything before it is
#in list[0] and since there's nothing after, store nothing in list[1]. So list[0] has all the cost values
#challenge: do in one line
clean[cost] = clean[cost].map(lambda x: x.lstrip('£').rstrip('$')) #took 2 lines to do this
clean[cost] = clean[cost].astype('float64')
#money_analysis = pd.to_numeric(money_fix)

count      2127.000000
mean      24067.339972
std      146860.665559
min           0.000000
25%        1280.000000
50%        1884.010000
75%        2321.305000
max      999999.000000
Name: COST (£) charged to Wellcome (inc VAT when charged), dtype: float64

In [12]:
clean[cost].describe()

count      2127.000000
mean      24067.339972
std      146860.665559
min           0.000000
25%        1280.000000
50%        1884.010000
75%        2321.305000
max      999999.000000
Name: COST (£) charged to Wellcome (inc VAT when charged), dtype: float64

In [7]:
publisher = clean.groupby('Publisher')['Article title']
print("The top 5 Publishers and the number of articles they have: \n{}".format(publisher.count().sort_values(ascending=False)[:5]))

The top 5 Publishers and the number of articles they have: 
Publisher
Elsevier                   408
PLoS                       307
Wiley                      244
Oxford University Press    167
BioMed Central              95
Name: Article title, dtype: int64


In [11]:
print(clean.loc[clean[cost]<6000.0].groupby('Publisher')[cost].describe())

                                                    count         mean  \
Publisher                                                                
ACS                                                  30.0  1262.137333   
Aga institute                                         1.0   238.080000   
American Physiological Society                        1.0  1288.000000   
American Psychological Association                   16.0  2836.150000   
American Society for Biochemistry and Molecular...   72.0  1372.228333   
American Society for Microbiology                    24.0  1718.043333   
American Society of Hematology                        9.0  1402.984444   
American association of immunologists                 2.0  2571.540000   
American college of chest physicians                  1.0  2383.940000   
American physiological society                        7.0  1363.344286   
American public health association                    1.0  1422.250000   
American society for investigative pat

In [9]:
#save at the end and before any data analysis begins
clean.reset_index()

,index,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,0,NaN,Cambridge University Press,Psychological medicine,Reduced parahippocampal cortical thickness in ...,0.00
1,1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,2381.04
2,2,23043264 PMC3506128,ACS,J med chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",642.56
3,3,23438330 PMC3646402,ACS,J med chem,Orvinols with mixed kappa/mu opioid receptor a...,669.64
4,4,23438216 PMC3601604,ACS,J org chem,Regioselective opening of myo-inositol orthoes...,685.88
5,5,PMC3579457,ACS,Journal of medicinal chemistry,Comparative Structural and Functional Studies ...,2392.20
6,6,PMC3709265,ACS,Journal of proteome research,Mapping Proteolytic Processing in the Secretom...,2367.95
7,7,23057412 PMC3495574,ACS,Mol pharm,Quantitative silencing of EGFP reporter gene b...,649.33
8,8,PMCID: PMC3780468,ACS,Acs chemical biology,A Novel Allosteric Inhibitor of the Uridine Di...,1294.59
9,9,PMCID: PMC3621575,ACS,Acs chemical biology,Chemical proteomic analysis reveals the drugab...,1294.78
